# Import

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer


# Data exploring

In [3]:
#data exploring
df = pd.read_csv('DATI.csv')

df

,timestamp,accX,accY,accZ,gyroX,gyroY,gyroZ,Label
0,1.732189e+09,-5.527879,-4.858718,-2.166334,-0.966667,-3.166667,-0.966667,"['sotto', 'fermo', 'fermo', 'fermo', 'sinistra..."
1,1.732189e+09,-5.200716,-5.413599,-2.259820,-0.366667,-2.866667,-1.400000,"['sotto', 'indietro', 'fermo', 'fermo', 'sinis..."
2,1.732189e+09,-3.524709,-5.594171,-1.247677,0.600000,-1.800000,-2.133333,"['fermo', 'indietro', 'fermo', 'fermo', 'sinis..."
3,1.732189e+09,-0.023709,-6.387939,1.155626,0.833333,-0.166667,-2.433333,"['fermo', 'indietro', 'fermo', 'fermo', 'fermo..."
4,1.732189e+09,2.153291,-6.756805,2.129320,0.600000,1.000000,-2.400000,"['fermo', 'indietro', 'fermo', 'fermo', 'fermo..."
...,...,...,...,...,...,...,...,...
40379,1.732189e+09,6.151457,-13.349115,13.103255,2.633333,2.133333,0.666667,"['sopra', 'indietro', 'destra', 'sinistra brac..."
40380,1.732189e+09,3.876457,-8.463914,11.822385,1.733333,0.833333,0.333333,"['fermo', 'indietro', 'destra', 'sinistra brac..."
40381,1.732189e+09,2.689457,-6.649928,11.716661,1.266667,0.133333,0.300000,"['fermo', 'indietro', 'destra', 'sinistra brac..."
40382,1.732189e+09,0.434457,-7.581928,12.356489,-0.266667,-0.066667,0.533333,"['fermo', 'indietro', 'destra', 'fermo', 'ferm..."


# Data cleaning

In [4]:
# rimuoviamo le label in eccesso tenendo solo le prime 3 righe
def trova_terza(str):
    for _ in range(3):
        idx = str.rfind(',')
        str = str[:idx]
    str = str + ']'
    print(str)

    return str

In [5]:
df['Label'] = df['Label'].apply(trova_terza)

['sotto', 'fermo', 'fermo']
['sotto', 'indietro', 'fermo']
['fermo', 'indietro', 'fermo']
['fermo', 'indietro', 'fermo']
['fermo', 'indietro', 'fermo']
['sopra', 'indietro', 'fermo']
['sopra', 'indietro', 'fermo']
['sopra', 'fermo', 'fermo']
['sopra', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'indietro', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'indietro', 'fermo']
['fermo', 'indietro', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'fermo', 'fermo']
['fermo', 'indietro', 'fermo']
['fermo', 'indietr

# aggregamento dati

In [6]:

def aggrega(df, colonna, batch_size=10):
    """
    Assegna la label più frequente ogni blocco di 10
    da specificare il df e la ['']
    """
    
    lista = []  
    
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i + batch_size]  
        
        labels = batch[colonna]  #label del batch
        counter = Counter(labels)     #trova la label più frequente 
        most_common_label = counter.most_common(1)[0][0]
        
        lista.extend([most_common_label] * len(batch))
    
    return pd.Series(lista, index=df.index)

In [7]:
df['Label'] = aggrega(df, colonna='Label')

In [8]:
df

,timestamp,accX,accY,accZ,gyroX,gyroY,gyroZ,Label
0,1.732189e+09,-5.527879,-4.858718,-2.166334,-0.966667,-3.166667,-0.966667,"['fermo', 'indietro', 'fermo']"
1,1.732189e+09,-5.200716,-5.413599,-2.259820,-0.366667,-2.866667,-1.400000,"['fermo', 'indietro', 'fermo']"
2,1.732189e+09,-3.524709,-5.594171,-1.247677,0.600000,-1.800000,-2.133333,"['fermo', 'indietro', 'fermo']"
3,1.732189e+09,-0.023709,-6.387939,1.155626,0.833333,-0.166667,-2.433333,"['fermo', 'indietro', 'fermo']"
4,1.732189e+09,2.153291,-6.756805,2.129320,0.600000,1.000000,-2.400000,"['fermo', 'indietro', 'fermo']"
...,...,...,...,...,...,...,...,...
40379,1.732189e+09,6.151457,-13.349115,13.103255,2.633333,2.133333,0.666667,"['sotto', 'indietro', 'sinistra']"
40380,1.732189e+09,3.876457,-8.463914,11.822385,1.733333,0.833333,0.333333,"['fermo', 'indietro', 'destra']"
40381,1.732189e+09,2.689457,-6.649928,11.716661,1.266667,0.133333,0.300000,"['fermo', 'indietro', 'destra']"
40382,1.732189e+09,0.434457,-7.581928,12.356489,-0.266667,-0.066667,0.533333,"['fermo', 'indietro', 'destra']"


# Data cleaning PT 2

In [9]:
# la label è salvata come una stringa unica, quindi la puliamo e dividiamo con strip
# che dividerà in elementi di una lista
df['Label'] = df['Label'].apply(lambda x: x.replace('[',''))
df['Label'] = df['Label'].apply(lambda x: x.replace(']',''))
df['Label'] = df['Label'].apply(lambda x: x.replace('\'',''))

df

,timestamp,accX,accY,accZ,gyroX,gyroY,gyroZ,Label
0,1.732189e+09,-5.527879,-4.858718,-2.166334,-0.966667,-3.166667,-0.966667,"fermo, indietro, fermo"
1,1.732189e+09,-5.200716,-5.413599,-2.259820,-0.366667,-2.866667,-1.400000,"fermo, indietro, fermo"
2,1.732189e+09,-3.524709,-5.594171,-1.247677,0.600000,-1.800000,-2.133333,"fermo, indietro, fermo"
3,1.732189e+09,-0.023709,-6.387939,1.155626,0.833333,-0.166667,-2.433333,"fermo, indietro, fermo"
4,1.732189e+09,2.153291,-6.756805,2.129320,0.600000,1.000000,-2.400000,"fermo, indietro, fermo"
...,...,...,...,...,...,...,...,...
40379,1.732189e+09,6.151457,-13.349115,13.103255,2.633333,2.133333,0.666667,"sotto, indietro, sinistra"
40380,1.732189e+09,3.876457,-8.463914,11.822385,1.733333,0.833333,0.333333,"fermo, indietro, destra"
40381,1.732189e+09,2.689457,-6.649928,11.716661,1.266667,0.133333,0.300000,"fermo, indietro, destra"
40382,1.732189e+09,0.434457,-7.581928,12.356489,-0.266667,-0.066667,0.533333,"fermo, indietro, destra"


In [10]:
df['Label'] = df['Label'].apply(lambda x: x.split(','))
df['Label']
#trasfroma in lista

0           [fermo,  indietro,  fermo]
1           [fermo,  indietro,  fermo]
2           [fermo,  indietro,  fermo]
3           [fermo,  indietro,  fermo]
4           [fermo,  indietro,  fermo]
                     ...              
40379    [sotto,  indietro,  sinistra]
40380      [fermo,  indietro,  destra]
40381      [fermo,  indietro,  destra]
40382      [fermo,  indietro,  destra]
40383      [fermo,  indietro,  destra]
Name: Label, Length: 40384, dtype: object

In [11]:
# come scelta progettuale ho eliminato le label non necessarie:

""" 
dato che facciamo una classificazione multi-classe dobbiamo dividere tutte le label e non ci serve sapere che dei
sensori sono in stato fermo, ma solo che il moviemento che ci interessa, quindi avanti/indietro/destra/ecc...
"""
def trasforma(lista):
    contatore = 0
    for elemento in lista:
        if elemento.strip() == 'fermo':
            lista.remove(elemento)
            contatore += 1
    
    if contatore == 3:
        elemento = 'fermo'
        return elemento
    
    return lista

In [12]:
df['Label'] = df['Label'].apply(trasforma)
df

,timestamp,accX,accY,accZ,gyroX,gyroY,gyroZ,Label
0,1.732189e+09,-5.527879,-4.858718,-2.166334,-0.966667,-3.166667,-0.966667,[ indietro]
1,1.732189e+09,-5.200716,-5.413599,-2.259820,-0.366667,-2.866667,-1.400000,[ indietro]
2,1.732189e+09,-3.524709,-5.594171,-1.247677,0.600000,-1.800000,-2.133333,[ indietro]
3,1.732189e+09,-0.023709,-6.387939,1.155626,0.833333,-0.166667,-2.433333,[ indietro]
4,1.732189e+09,2.153291,-6.756805,2.129320,0.600000,1.000000,-2.400000,[ indietro]
...,...,...,...,...,...,...,...,...
40379,1.732189e+09,6.151457,-13.349115,13.103255,2.633333,2.133333,0.666667,"[sotto, indietro, sinistra]"
40380,1.732189e+09,3.876457,-8.463914,11.822385,1.733333,0.833333,0.333333,"[ indietro, destra]"
40381,1.732189e+09,2.689457,-6.649928,11.716661,1.266667,0.133333,0.300000,"[ indietro, destra]"
40382,1.732189e+09,0.434457,-7.581928,12.356489,-0.266667,-0.066667,0.533333,"[ indietro, destra]"


In [13]:
df['Label'].value_counts()
#controllo se ci sono label vuote (no)

Label
[ fermo]                         7880
[ indietro]                      6590
[sopra,  indietro]               2930
[sotto,  indietro,  sinistra]    2870
[sopra,  fermo]                  2440
[sotto,  indietro]               2370
[ indietro,  sinistra]           2330
[ indietro,  destra]             1804
[ fermo,  sinistra]              1800
[sotto,  fermo]                  1480
[sopra,  indietro,  destra]      1230
[ avanti]                        1100
[sotto,  indietro,  destra]      1070
[sopra,  indietro,  sinistra]     890
[sotto,  sinistra]                680
[sopra,  avanti,  sinistra]       430
[sopra,  avanti]                  420
[ fermo,  destra]                 380
[ avanti,  sinistra]              360
[sopra,  sinistra]                320
[sotto,  avanti,  sinistra]       210
[sotto,  destra]                  190
[sotto,  avanti]                  170
[sopra,  avanti,  destra]         140
[ avanti,  destra]                120
[sopra,  destra]                  120
[sotto

In [14]:
# trasformazione label in binarie, per poi fornirle al modello
mlb = MultiLabelBinarizer()
binary_labels = mlb.fit_transform(df['Label'])
print(binary_labels[0])

[0 0 0 1 0 0 0]


In [15]:
#mostra le classi corrispondenti
print(mlb.classes_) 

[' avanti' ' destra' ' fermo' ' indietro' ' sinistra' 'sopra' 'sotto']


# aggiuntiva per migliorare il training
non runnate a meno che sicuri

In [16]:
df2 = df.drop('timestamp', axis = 1)
df2 = df2.drop('gyroX', axis = 1)
df2 = df2.drop('gyroY', axis = 1)
df2 = df2.drop('gyroZ', axis = 1)


In [17]:
df2

,accX,accY,accZ,Label
0,-5.527879,-4.858718,-2.166334,[ indietro]
1,-5.200716,-5.413599,-2.259820,[ indietro]
2,-3.524709,-5.594171,-1.247677,[ indietro]
3,-0.023709,-6.387939,1.155626,[ indietro]
4,2.153291,-6.756805,2.129320,[ indietro]
...,...,...,...,...
40379,6.151457,-13.349115,13.103255,"[sotto, indietro, sinistra]"
40380,3.876457,-8.463914,11.822385,"[ indietro, destra]"
40381,2.689457,-6.649928,11.716661,"[ indietro, destra]"
40382,0.434457,-7.581928,12.356489,"[ indietro, destra]"


In [18]:
# Caricamento e suddivisione dei dati
x = df2.drop('Label', axis  = 1)
y = binary_labels

# train test split

In [ ]:
# Caricamento e suddivisione dei dati
x = df.drop('Label', axis  = 1)
y = binary_labels

In [19]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# normalizzazione

In [20]:
scaler = MinMaxScaler(feature_range=(0, 1))
x_test  = scaler.fit_transform(x_test)
x_train  = scaler.transform(x_train)

In [21]:
print(x_train.shape)
y_train.shape

(32307, 3)


(32307, 7)

# convertire i dati per il modello

In [22]:
sequence_length = 10 #sequenza degli istanti temporali

def create_sequences(train, sequence_length):
    
    seq_temp = []
    for i in range(len(train) - sequence_length + 1):
        seq_temp.append(train[i:i + sequence_length])
    return np.array(seq_temp)

x_train = create_sequences(x_train, sequence_length)
x_test = create_sequences(x_test, sequence_length)

print(x_train.shape)
x_test.shape

(32298, 10, 3)


(8068, 10, 3)

In [23]:
#scartiamo da y_test i dati in eccesso che non riescono a creare una sequenza intera
diff = len(y_test) - len(x_test)
print(diff)

if diff > 0:
    y_test = y_test[:-diff]

9


# addestramento

In [28]:
'''# Creazione del modello LSTM
model = Sequential()
model.add(LSTM(64, input_shape=(sequence_length, 7), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(32))
model.add(Dropout(0.2))
model.add(Dense(7, activation='sigmoid'))  


# Compilazione del modello
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Allenamento
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)
'''

"# Creazione del modello LSTM\nmodel = Sequential()\nmodel.add(LSTM(64, input_shape=(sequence_length, 7), return_sequences=True))\nmodel.add(Dropout(0.2))\nmodel.add(LSTM(32))\nmodel.add(Dropout(0.2))\nmodel.add(Dense(7, activation='sigmoid'))  \n\n\n# Compilazione del modello\nmodel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])\n\n# Allenamento\nmodel.fit(x_train, y_train, epochs=5, batch_size=32, validation_split=0.2)\n"

In [36]:
# Creazione del modello LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(sequence_length, 3), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(7, activation='sigmoid'))  


# Compilazione del modello
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Allenamento
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)



Epoch 1/10
808/808 ━━━━━━━━━━━━━━━━━━━━ 27s 26ms/step - accuracy: 0.4573 - loss: 0.5137 - val_accuracy: 0.6621 - val_loss: 0.3981
Epoch 2/10
808/808 ━━━━━━━━━━━━━━━━━━━━ 27s 33ms/step - accuracy: 0.6702 - loss: 0.3802 - val_accuracy: 0.6895 - val_loss: 0.3464
Epoch 3/10
808/808 ━━━━━━━━━━━━━━━━━━━━ 23s 29ms/step - accuracy: 0.6818 - loss: 0.3458 - val_accuracy: 0.7033 - val_loss: 0.3313
Epoch 4/10
808/808 ━━━━━━━━━━━━━━━━━━━━ 20s 24ms/step - accuracy: 0.6768 - loss: 0.3267 - val_accuracy: 0.6302 - val_loss: 0.3062
Epoch 5/10
808/808 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - accuracy: 0.6334 - loss: 0.3010 - val_accuracy: 0.6636 - val_loss: 0.2835
Epoch 6/10
808/808 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - accuracy: 0.6505 - loss: 0.2903 - val_accuracy: 0.6111 - val_loss: 0.2835
Epoch 7/10
808/808 ━━━━━━━━━━━━━━━━━━━━ 18s 22ms/step - accuracy: 0.6326 - loss: 0.2858 - val_accuracy: 0.6432 - val_loss: 0.2818
Epoch 8/10
808/808 ━━━━━━━━━━━━━━━━━━━━ 18s 23ms/step - accuracy: 0.6458 - loss: 0.2816 - 

In [32]:
from sklearn.metrics import accuracy_score
# Valutazione

predizioni = model.predict(x_test)
soglia_corretto = 0.7
predicted_labels = (predizioni > soglia_corretto)

accuracy = accuracy_score(y_test, predicted_labels)

253/253 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


In [33]:
print(f"Accuracy: {accuracy}")

Accuracy: 0.48574615765989093
